In [ ]:
# Colab+VITS Fast（使用前查看原作者是否做了修改）

# 配置Conda
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py38_23.3.1-0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_23.3.1-0-Linux-x86_64.sh
!bash Miniconda3-py38_23.3.1-0-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.insert(0, "/usr/local/lib/python3.8/site-packages/") # 加入环境变量

!conda create -n vits python=3.8.1 -y # 创建独立环境
!activate vits
!python -V

# 下载项目并安装依赖
!git clone https://github.com/Plachtaa/VITS-fast-fine-tuning.git
%cd VITS-fast-fine-tuning
!pip install -r requirements.txt

# 编译monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..

# 下载预模型
!wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/sampled_audio4ft_v2.zip
!unzip sampled_audio4ft_v2.zip

# 创建目录
!mkdir raw_audio
!mkdir denoised_audio
!mkdir pretrained_models
!mkdir custom_character_voice
!mkdir segmented_character_voice

In [ ]:
# 选择预训练模型
PRETRAINED_MODEL = "CJ" #@param ["CJ","CJE"]
if PRETRAINED_MODEL == "CJ": # 中日双语
    !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/D_0-p.pth -O ./pretrained_models/D_0.pth
    !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/G_0-p.pth -O ./pretrained_models/G_0.pth
    !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/config.json -O ./configs/finetune_speaker.json
elif PRETRAINED_MODEL == "CJE": # 中日英三语
    !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/D_trilingual.pth -O ./pretrained_models/D_0.pth
    !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/G_trilingual.pth -O ./pretrained_models/G_0.pth
    !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/configs/uma_trilingual.json -O ./configs/finetune_speaker.json

# 挂载Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# 查看音频文件
!ls -l -F /content/VITS-fast-fine-tuning/custom_character_voice

# 从Google Drive拷贝压缩包
!cp /content/drive/MyDrive/test.zip /content/VITS-fast-fine-tuning/custom_character_voice/test.zip
!unzip ./custom_character_voice/test.zip -d ./custom_character_voice/
# !cp /content/drive/MyDrive/test.wav ./raw_audio/ # 长音频

# 降噪
!python scripts/denoise_audio.py

# 分割标注音频文件
!python scripts/short_audio_transcribe.py --languages "CJ" --whisper_size large
# !python scripts/long_audio_transcribe.py --languages "CJ" --whisper_size large

# （可能的）重采样
!python scripts/resample.py

# 样本少于100条使用辅助训练数据，反之不使用
!python preprocess_v2.py --languages "CJ"
# !python preprocess_v2.py --add_auxiliary_data True --languages "CJ"

# 训练
Maximum_epochs = "50" #@param [50, 100, 150]
!python finetune_speaker_v2.py -m "./OUTPUT_MODEL" --max_epochs "{Maximum_epochs}" --drop_speaker_embed True

# 测试
!cp ./configs/modified_finetune_speaker.json ./finetune_speaker.json
!python VC_inference.py --model_dir ./OUTPUT_MODEL/G_latest.pth --share True

# 下载模型
!python scripts/rearrange_speaker.py 
!python scripts/download_model.py

# 备份模型
!python scripts/rearrange_speaker.py
!cp ./G_latest.pth /content/drive/MyDrive/test/G_latest.pth
!cp ./finetune_speaker.json /content/drive/MyDrive/test/finetune_speaker.json

# 查看模型
!ls -l -F ./OUTPUT_MODEL

# 删除音频文件及模型
!rm -rf ./custom_character_voice/*
!rm -rf ./raw_audio/*
!rm -rf ./denoised_audio/*
!rm -rf ./segmented_character_voice/*
!rm -rf long_character_anno.txt
!rm -rf short_character_anno.txt